<a href="https://colab.research.google.com/github/LinoMota/learning-ts/blob/main/C%C3%B3pia_de_notebook_final_radical-roda-ae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Videos selecionados e seus pontos de corte

```
1)
# [COZY FM] - and then...
# youtube.com/watch?v=LLLJ9rznmYI
[10, 16, 20, 21,  22, 26, 28, 34, 40, 45, 49, 52, 54, 58, 69]

2)
# villa [sun.set x aimless]
# youtube.com/watch?v=bECAq7hu1T8
[2, 4, 5, 7, 11, 13, 16, 17, 18, 20, 21, 22, 26, 27, 31,
    36, 39, 40, 42, 44, 48, 50, 51,  52, 53, 55, 59, 64]

3)
# aebleo - aerodynamics
# youtube.com/watch?v=UDNvOwgkUoo
[4, 7, 8, 10, 11, 13, 15, 16, 18, 19, 20, 21, 22, 26, 29, 32, 33,
    35, 36, 38, 40, 42, 49, 50, 53, 55, 56, 57, 58, 61, 62, 65, 66]

4)
# cat naps - would you miss me
# youtube.com/watch?v=9MO8ExMyla4
[3, 4, 10, 12, 15, 18, 24, 27, 30, 31, 33, 34, 35, 38,
    40, 45, 47, 49, 53, 55, 56, 58, 63, 65, 68, 72]

5)
# e e v e e - romance
# youtube.com/watch?v=XywZDRcdjWM
[3, 6, 10, 11, 13, 20, 23, 26, 27, 29, 32, 36, 38, 40,  44,
    46, 47, 51, 52, 53, 55, 56, 58, 60, 61, 65, 66, 68, 71, 75]
```

In [ ]:
import numpy as np
    import cv2 as cv
import math

from numpy.ma.core import zeros
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow


In [ ]:
def is_image_on_second(c_frame, seconds, video_framerate=30):
  return c_frame % (seconds * video_framerate) == 0

In [ ]:
def histograma_global(frame):
  r = zeros(256, dtype=int)
  g = zeros(256, dtype=int)
  b = zeros(256, dtype=int)

  for i in range(0, frame.shape[0]):
    for j in range(0, frame.shape[1]):
      r[frame[i][j][2]] += 1
      g[frame[i][j][1]] += 1
      b[frame[i][j][0]] += 1
      

  return r+g+b

In [ ]:
def interseccao_hist(frame1, frame2):
  hI1 = histograma_global(frame1) 
  hI2 = histograma_global(frame2)

  numerador = 0
  for i in range(0, len(hI2)):
    numerador += min(hI1[i], hI2[i])
  
  interseccao = 1 - (numerador / sum(hI1))
  return interseccao


In [ ]:
def normalizar_vetor(vetor):
  sum = 0
  for e in vetor:
      sum += e ** 2

  if sum == 0:
      return [0.0] * len(vetor)

  vet = []

  for e in vetor:
      vet.append(e / math.sqrt(sum))
  
  
  
  return vet 

In [ ]:
def bic(image):
  image_cinza = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
  interior = zeros(256, dtype = int)
  borda = zeros(256, dtype = int)

  for i in range(1, image.shape[0] - 1):
      for j in range(1, image.shape[0] - 1 ):

          atual = image_cinza[i][j]
          if(atual == image_cinza[i - 1][j] and atual == image_cinza[i][j - 1] and
              atual == image_cinza[i + 1][j] and atual == image_cinza[i][j + 1]):
              interior[atual] += 1
          else:
              borda[atual] += 1 


  return normalizar_vetor(normalizar_vetor(interior) + normalizar_vetor(borda))

In [ ]:
def distancia_euclid(frame1, frame2):
  sum = 0
  
  hI1 = bic(frame1) 
  hI2 = bic(frame2)

  for i in range(0, len(hI1)):
      sum += (hI2[i] - hI1[i])**2
  sum = math.sqrt(sum)

  return sum

In [ ]:
def compara_imagem(image1, image2, key="histograma"):
  if key =="histograma":
    mult_factor = 100
    return interseccao_hist(image1, image2) * mult_factor
  if key =="bic":
    mult_factor_bic = 10
    return distancia_euclid(image1, image2) * mult_factor_bic

In [ ]:
def gerar_particoes(frame, grid=5):
  # print("ESTOU GERANDO PARTICOES")

  img_alt = int(np.shape(frame)[1] / grid)
  img_lar = int(np.shape(frame)[0] / grid)

  imgs = []


  for i in range(0, grid):
      for j in range(0, grid):

        lim_inf_alt = math.floor(img_alt * i)
        lim_sup_alt = math.floor(img_alt * (i+1))

        lim_inf_lar = math.floor(img_lar * j)
        lim_sup_lar = math.floor(img_lar * (j+1))

        imgs.append(frame[lim_inf_lar:lim_sup_lar, lim_inf_alt:lim_sup_alt])
  return imgs


In [ ]:

def compara_grid(grid1, grid2, key="histograma"):
  grid = 5
  it_size = grid * grid

  
  soma = 0
  val = 0
  for i in range(0, it_size):
    # cv2_imshow(grid1[i])
    # cv2_imshow(grid2[i])
    # print(f'comparando parte {i+1} diff nessa parte foi de {compara_imagem(grid1[i], grid2[i], key=key)}')
    val = compara_imagem(grid1[i], grid2[i], key=key)
    # print(f'compara imagem  {val}')
    soma += val


#   print(f'soma {soma}', f'it_size {it_size}')  
  return soma / it_size


In [ ]:
def processar_video(video_filename, fps=30):
  vid = cv.VideoCapture(f'./{video_filename}')
  totalFrames = vid.get(cv.CAP_PROP_FRAME_COUNT)
  limiar = 35
  limiar_bic = 7

  cenas_de_corte = {
      "bic": [],
      "bic_local": [],
      "histograma": [],
      "histograma_local": []
  }

  fps = int(vid.get(5))
  w = fps / 2

  while vid.isOpened():
    tocando, frame = vid.read()

    if not tocando:
      break
    
    cont_frame = int(vid.get(cv.CAP_PROP_POS_FRAMES))
    if( cont_frame % w == 0):


      frame_chave = frame
      calc_frame = cont_frame + (2 * w)

      if(calc_frame > totalFrames):
        break;

      if(calc_frame < totalFrames):
        vid.set(cv.CAP_PROP_POS_FRAMES, calc_frame)
        
        tocando, frame_2 = vid.read()

        # print(f'seguntdo atual {(cont_frame - w)  / fps} -- segundo seg {(calc_frame  - w) / fps}')
        
        if(compara_imagem(frame_chave, frame_2, key="histograma") > limiar):
          cenas_de_corte["histograma"].append(cont_frame)

        if(compara_imagem(frame_chave, frame_2, key="bic") > limiar_bic):
            cenas_de_corte["bic"].append(cont_frame)


        grid1 = gerar_particoes(frame_chave)
        grid2 = gerar_particoes(frame_2)


        if(compara_grid(grid1, grid2, key="histograma") > limiar):
            cenas_de_corte["histograma_local"].append(cont_frame)


        if(compara_grid(grid1, grid2, key="bic") > limiar_bic):
            cenas_de_corte["bic_local"].append(cont_frame)
  
  vid.release()

  return cenas_de_corte

In [ ]:
import json

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

video_nums = 10
videos_process_arr = []


for i in range(8, video_nums):
  
  filename= f'./video_{i+1}'
  print(f'Obtendo pontos de cortes para o video {i+1} : {filename}.mp4')

  

  result_dict = processar_video(f'./{filename}.mp4')
  videos_process_arr.append(result_dict)
#   fps = video.get(cv2.cv.CV_CAP_PROP_FPS)

  
  with open(f'{filename}_results.json', 'w') as file:
     file.write(json.dumps(result_dict, cls=NpEncoder))


Obtendo pontos de cortes para o video 9 : ./video_9.mp4
Obtendo pontos de cortes para o video 10 : ./video_10.mp4


In [ ]:
def mostra_cortes(filename, video_filename):
    vid = cv.VideoCapture(f'./{video_filename}')
    
    
    fps = int(vid.get(5))

    f = open(filename)
    data = json.load(f)

    
    print(data['bic'] / fps)

mostra_cortes('./video_2_results.json', './video_2.mp4')

FileNotFoundError: ignored

In [ ]:

fps = 30
w = fps / 2


vid = cv.VideoCapture('./lesgo.mp4')
totalFrames = vid.get(cv.CAP_PROP_FRAME_COUNT)

limiar = 15
limiar_bic = 7

cenas_cortes = []

while vid.isOpened():
  tocando, frame = vid.read()

  if not tocando:
    print("Video acabou ou nao esta mais disponivel!")
    break
  
  cont_frame = int(vid.get(cv.CAP_PROP_POS_FRAMES))
  if( cont_frame % w == 0):

    frame_chave = frame
    calc_frame = cont_frame + (2 * w + 1)

    if(calc_frame > totalFrames):
      break;

    # 2w + 1 -> proximo frame chave
    if(calc_frame < totalFrames):
      vid.set(cv.CAP_PROP_POS_FRAMES, calc_frame)
      
      tocando, frame_2 = vid.read()

      # if(compara_imagem(frame, frame_2, key="histograma")  > limiar):
      #   cenas_cortes.append(frame_chave)
      # dist = compara_imagem(frame, frame_2, key="bic")
      print(f'{calc_frame}, {cont_frame}')

      dist = compara_grid(frame, frame_2, key="bic")  

      print(dist)

      # if(dist  > limiar):
      #   cenas_cortes.append(frame_chave)
      
      
  
      if( dist > limiar_bic):
        # cv2_imshow(frame)
        cenas_cortes.append((frame_chave, cont_frame))
  


vid.release()
# vid.destroyAllWindows()

46.0, 15
11.447816968063012
91.0, 60
13.779958250969392
136.0, 105
9.691964037762958
181.0, 150
3.1436837890484166
226.0, 195
13.665496938352227
271.0, 240
13.45041243824151
316.0, 285
13.336833304768271
361.0, 330
13.781946182880834
406.0, 375
13.424198293086642
451.0, 420
13.423650545433082
496.0, 465
13.483887228714124
541.0, 510
13.580113940847435
586.0, 555
1.7488264813526249
631.0, 600
2.219352131666933
676.0, 645
0.9422499846772192
721.0, 690
13.668054258433164
766.0, 735
13.90458407777401
811.0, 780
13.855527966531577
856.0, 825
14.14213562373095
901.0, 870
12.679995755680306
946.0, 915
12.753229948083643
991.0, 960
7.29870803630786
1036.0, 1005
13.342059103268355
1081.0, 1050
13.590857945335518
1126.0, 1095
13.81911150088863
1171.0, 1140
13.482136999599186
1216.0, 1185
13.984453385930125
1261.0, 1230
5.823352345282794
1306.0, 1275
12.862687090722462
1351.0, 1320
13.706372066928314
1396.0, 1365
13.327328619283806
1441.0, 1410
6.4697803217137
1486.0, 1455
14.079294872762556
1531

In [ ]:
print(cenas_cortes)


[(array([[[208, 151, 136],
        [208, 151, 136],
        [208, 151, 136],
        ...,
        [213, 153, 129],
        [213, 153, 129],
        [213, 153, 129]],

       [[210, 153, 138],
        [210, 153, 138],
        [210, 153, 138],
        ...,
        [216, 156, 132],
        [216, 156, 132],
        [216, 156, 132]],

       [[211, 154, 141],
        [211, 154, 141],
        [210, 153, 140],
        ...,
        [217, 157, 133],
        [217, 157, 133],
        [217, 157, 133]],

       ...,

       [[ 18,  18,  18],
        [ 31,  31,  31],
        [ 19,  19,  19],
        ...,
        [ 34,  34,  34],
        [  9,   9,   9],
        [  0,   0,   0]],

       [[ 13,  13,  13],
        [ 30,  30,  30],
        [ 18,  18,  18],
        ...,
        [ 32,  32,  32],
        [  6,   6,   6],
        [  0,   0,   0]],

       [[ 12,  12,  12],
        [ 30,  30,  30],
        [ 18,  18,  18],
        ...,
        [ 31,  31,  31],
        [  5,   5,   5],
        [  0,   0,   0